In [ ]:
import torch.utils.data
from torch.utils.data.dataset import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as tvm
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
import sklearn.metrics as skms

In [ ]:
path = 'new_images/' # custom data

myL = os.listdir(path)
numCl = len(myL)

In [ ]:
myL # confirming the labels

In [ ]:
custom_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply(torch.nn.ModuleList([transforms.ColorJitter()]), p=0.1), # to change color a little bit. When image does not have perect lighting
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

all_img = datasets.ImageFolder(path, transform=custom_transform)
len(all_img)

In [ ]:
# all_img.class_to_idx

In [ ]:
total_count = len(all_img)
train_count = int(0.70 * total_count)
valid_count = int(0.15* total_count)
test_count = total_count - train_count - valid_count # 15 percent for tesing
train_set, valid_set, test_set = torch.utils.data.random_split(all_img, (train_count, valid_count,test_count))


In [ ]:
# Loaders to Train, Validate and Test 
tr_loader = torch.utils.data.DataLoader(train_set, batch_size=100,shuffle=True)
tv_loader = torch.utils.data.DataLoader(valid_set, batch_size=100,shuffle=True)
ts_loader = torch.utils.data.DataLoader(test_set, batch_size=100,shuffle=True)

In [ ]:
# Can be used to plot when debugging

# def plot_images(images, labels,normalize = True):

#     n_images = len(images)

#     rows = int(np.sqrt(n_images))
#     cols = int(np.sqrt(n_images))

#     fig = plt.figure(figsize = (15, 15))

#     for i in range(rows*cols):

#         ax = fig.add_subplot(rows, cols, i+1)
        
#         image = images[i]
        
#         if normalize:
#             image = normalize_image(image)

#         ax.imshow(image.permute(1, 2, 0).cpu().numpy())
#         label = labels[i]
#         ax.set_title(label)
#         ax.axis('off')

In [ ]:
model = tvm.resnet50(pretrained = True) # importing pretrained resnet 50 model

inputFeatures = model.fc.in_features 
outputClasses = 10 # convert output into 10 classes.

fc = nn.Linear(inputFeatures, outputClasses)
model.fc = fc

In [ ]:
model   # Just to view it

In [ ]:
# Optimizer and Scheduler values not tested thoroughly. Picked a good estimate based off of general choices
optimizer = optim.Adam(model.parameters(), lr=3e-4) 
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
params = {'batch_size': 32, 'num_workers': 8}
num_epochs = 10
num_classes = 10

In [ ]:

# Code for training taken and modified from https://towardsdatascience.com/bird-by-bird-using-deep-learning-4c0fa81365d7.

# loop over epochs
for epoch in range(num_epochs):
    
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    # train the model
    model.train()
    train_loss = list()
    train_acc = list()
    for batch in tr_loader:
        x, y = batch
        
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        
        optimizer.zero_grad()
        # predict bird species
        y_pred = model(x)
        # calculate the loss
        loss = F.cross_entropy(y_pred, y)
        # backprop & update weights
        loss.backward()
        optimizer.step()
        # calculate the accuracy
        acc = skms.accuracy_score([val.item() for val in y], [val.item() for val in y_pred.argmax(dim=-1)])
        
        train_loss.append(loss.item())
        train_acc.append(acc)

        print('Train Loss: {:.4f} Acc: {:.4f}'.format(train_loss[-1], train_acc[-1]))

                
    # validate the model
    model.eval()
    val_loss = list()
    val_acc = list()
    with torch.no_grad():
        for batch in tv_loader:
            x, y = batch
            x = x.to(DEVICE)
            y = y.to(DEVICE)
            # predict bird species
            y_pred = model(x)
            
            # calculate the loss
            loss = F.cross_entropy(y_pred, y)
            # calculate the accuracy
            acc = skms.accuracy_score([val.item() for val in y], [val.item() for val in y_pred.argmax(dim=-1)])
        val_loss.append(loss.item())
        val_acc.append(acc)
        print('Val Loss: {:.4f} Acc: {:.4f}'.format(val_loss[-1], val_acc[-1]))
    # adjust the learning rate
    scheduler.step()

torch.save(model.state_dict(), "birdmd1.pth") # Saved so we dont need to train it multiple times

# test the model
true = list()
pred = list()
with torch.no_grad():
    for batch in ts_loader:
        x, y = batch
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        y_pred = model(x)
        true.extend([val.item() for val in y])
        pred.extend([val.item() for val in y_pred.argmax(dim=-1)])
# calculate the accuracy 
test_accuracy = skms.accuracy_score(true, pred)
print('Test accuracy: {:.3f}'.format(test_accuracy))

In [ ]:
# y_pred

In [ ]:
## Testing


# class_names = ['01.Mourning Dove',
#  '02.Common Grackle',
#  '03.Cardinal',
#  '04.American Goldfinch',
#  '05.Blue Jay',
#  '06.White Breasted Nuthatch',
#  '07.House Sparrow',
#  '08.Red Bellied Woodpecker',
#  '09.Downy Woodpecker',
#  '10.Red Winged Black Bird']

# model.eval()
# for i in range(6):
#     strr = "ml/YoloV3/testbrd{}.jpg".format(str(i+1))
#     print(strr)
#     img = Image.open(strr).convert('RGB')
#     tr = custom_transform(img)
#     tr = tr.unsqueeze(0)

#     logits = model(tr)
#     pred_probab = nn.Softmax(dim=1)(logits)
#     y_pred = pred_probab.argmax(1)
#     print(f"Predicted class: {y_pred}")
#     print(class_names[np.argmax(pred_probab.detach().numpy())])
#     print( " ")
